# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
csv_path='../WeatherPy/city_weather_data.csv'
city_weather_df=pd.read_csv(csv_path)
city_weather_df.head()

,id,city,lat,lon,temp (F),humidity (%),cloudiness (%),wind speed (mph)
0,3424607,Tasiilaq,65.6145,-37.6368,32.16,77,21,5.53
1,3414079,Olafsvik,64.8945,-23.7142,44.29,77,92,18.86
2,4035715,Avarua,-21.2078,-159.7750,77.05,69,82,4.61
3,4030556,Rikitea,-23.1203,-134.9692,71.92,59,25,20.45
4,935215,Saint-Philippe,-21.3585,55.7679,74.52,81,66,12.53


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [4]:
# Store 'Lat' and 'Lng' into  locations 
locations = city_weather_df[["lat", "lon"]].astype(float)
humidity = city_weather_df["humidity (%)"].astype(float)

# Create a Humidity Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
ideal_df=city_weather_df[(city_weather_df["temp (F)"]>75)
                         &(city_weather_df["temp (F)"]<100)
                         &(city_weather_df["humidity (%)"]>70)
                        &(city_weather_df["humidity (%)"]<=100)
                        &(city_weather_df["wind speed (mph)"]<10)
                        &(city_weather_df["cloudiness (%)"]<25)]
ideal_df

,id,city,lat,lon,temp (F),humidity (%),cloudiness (%),wind speed (mph)
42,5848280,Kapaa,22.0752,-159.3190,79.48,76,20,1.01
44,5855927,Hilo,19.7297,-155.0900,76.46,76,1,8.05
49,934479,Grand Gaube,-20.0064,57.6608,77.38,74,15,8.99
72,3393115,Paracuru,-3.4100,-39.0306,78.12,89,16,9.55
206,3545981,Niquero,20.0431,-77.5828,76.98,82,20,3.91
296,3393692,Itarema,-2.9248,-39.9167,75.69,88,5,8.52
552,3389822,Salinópolis,-0.6136,-47.3561,77.79,87,3,9.04


In [6]:
#ideal_coords=ideal_df.assign(coords=[*zip(ideal_df.lat, ideal_df.lon)])
ideal_df["lat"].astype(str)
ideal_df["lon"].astype(str)

42     -159.319
44      -155.09
49      57.6608
72     -39.0306
206    -77.5828
296    -39.9167
552    -47.3561
Name: lon, dtype: object

In [10]:
ideal_df['coords']=ideal_df["lat"].astype(str) +","+ ideal_df["lon"].astype(str)

<ipython-input-10-ca086219016d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ideal_df['coords']=ideal_df["lat"].astype(str) +","+ ideal_df["lon"].astype(str)


In [11]:
ideal_df

,id,city,lat,lon,temp (F),humidity (%),cloudiness (%),wind speed (mph),coords
42,5848280,Kapaa,22.0752,-159.3190,79.48,76,20,1.01,"22.0752,-159.319"
44,5855927,Hilo,19.7297,-155.0900,76.46,76,1,8.05,"19.7297,-155.09"
49,934479,Grand Gaube,-20.0064,57.6608,77.38,74,15,8.99,"-20.0064,57.6608"
72,3393115,Paracuru,-3.4100,-39.0306,78.12,89,16,9.55,"-3.41,-39.0306"
206,3545981,Niquero,20.0431,-77.5828,76.98,82,20,3.91,"20.0431,-77.5828"
296,3393692,Itarema,-2.9248,-39.9167,75.69,88,5,8.52,"-2.9248,-39.9167"
552,3389822,Salinópolis,-0.6136,-47.3561,77.79,87,3,9.04,"-0.6136,-47.3561"


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
ideal_df["Hotel Name"]=""

In [22]:
# find the closest restaurant of each type to coordinates
# geocoordinates
target_radius = 5000
target_type = "hotel"

# set up a parameters dictionary
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "radius": target_radius,
    #"rankby": "distance",
    "type": target_type,
    "key": g_key,
}
# use iterrows to iterate through pandas dataframe
for index, row in ideal_df.iterrows():

    # get restaurant type from df
    ideal_city = row['city']
    coords = row['coords']

    # add keyword to params dict.  This is an assignment function.
    params['location'] = coords

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {ideal_city}, {coords}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest {target_type} in {ideal_city} is {results[1]['name']}.")
        
        ideal_df.loc[index, 'Hotel Name'] = results[1]['name']
        #types_df.loc[index, 'address'] = results[0]['vicinity']
        #types_df.loc[index, 'price_level'] = results[0]['price_level']
        #types_df.loc[index, 'rating'] = results[0]['rating']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Retrieving Results for Index 42: Kapaa, 22.0752,-159.319.
Closest hotel in Kapaa is Sheraton Kauai Coconut Beach Resort.
------------
Retrieving Results for Index 44: Hilo, 19.7297,-155.09.
Closest hotel in Hilo is Hilo Hawaiian Hotel.
------------
Retrieving Results for Index 49: Grand Gaube, -20.0064,57.6608.
Closest hotel in Grand Gaube is Veranda Paul et Virginie Hotel & Spa.
------------
Retrieving Results for Index 72: Paracuru, -3.41,-39.0306.
Closest hotel in Paracuru is Hotel Vento Brasil.
------------
Retrieving Results for Index 206: Niquero, 20.0431,-77.5828.
Closest hotel in Niquero is Religiosas Misioneras Siervas del Espíritu Santo.
------------
Retrieving Results for Index 296: Itarema, -2.9248,-39.9167.
Closest hotel in Itarema is Sea Lounge! Bar, Club & Restaurant.
------------
Retrieving Results for Index 552: Salinópolis, -0.6136,-47.3561.
Closest hotel in Salinópolis is Salinas Park Resort Maçarico AP101.
------------


In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
